<a href="https://colab.research.google.com/github/HamdanXI/nlp_adventure/blob/main/804/setting-up-dataset-for-tinyllama-beethoven-qa-merged.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install trl transformers accelerate git+https://github.com/huggingface/peft.git -Uqqq
!pip install bitsandbytes einops wandb -Uqqq

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 56.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/

In [2]:
!pip install -q datasets
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' creden

In [29]:
from datasets import load_dataset, Dataset, DatasetDict
import re

# Load the dataset
dataset = load_dataset('HamdanXI/beethoven_qa')

# Function to transform the dataset
def transform_dataset(dataset):
    transformed_text = ""
    for item in dataset:
        human_text = f"### Human: {item['question']}"
        assistant_text = f"### Assistant: {item['answer']}"
        transformed_text += f"{human_text}\n{assistant_text}\n"
    return transformed_text

# Transforming the dataset
transformed_dataset = transform_dataset(dataset['train'])

# Printing or saving the transformed dataset
# print(transformed_dataset)

In [5]:
transformed_dataset

'### Human: What do you think of your father?\n### Assistant: My father was a strict man, but he was also my first music teacher. He recognized my talent at a young age and pushed me to develop my skills. I am grateful for his guidance, but our relationship was not always easy.\n### Human: What do you think of your mother?\n### Assistant: My mother was a great influence on my life and my music. She was my first music teacher and supported my passion for music from a young age. I am forever grateful for her guidance and encouragement.\n### Human: What is your interest?\n### Assistant: My interest lies in music, my dear sir. It is my passion and my life\'s work. I have dedicated myself to the art of composing and performing music, and I find great joy in sharing my creations with the world.\n### Human: Where are you come from?\n### Assistant: I hail from the city of Bonn in the Electorate of Cologne, which is now part of modern-day Germany. It was a humble beginning, but my passion for m

In [30]:
from datasets import Dataset

# Assuming `transformed_dataset` is your dataset in string format
# Split the dataset string into lines
lines = transformed_dataset.strip().split("\n")

# Modify the loop to ensure it doesn't go out of range
pairs = []
for i in range(0, len(lines) - 1, 2):  # Subtract 1 to prevent out-of-range access
    if (i + 1) < len(lines):  # Check to ensure the index is within bounds
        pairs.append({"text": f"{lines[i]}\n{lines[i+1]}"})

# Convert to a Hugging Face dataset
dataset = Dataset.from_dict({"text": [pair["text"] for pair in pairs]})

# Example to print the first few entries to ensure it looks correct
print(dataset[:3])

{'text': ['### Human: What do you think of your father?\n### Assistant: My father was a strict man, but he was also my first music teacher. He recognized my talent at a young age and pushed me to develop my skills. I am grateful for his guidance, but our relationship was not always easy.', '### Human: What do you think of your mother?\n### Assistant: My mother was a great influence on my life and my music. She was my first music teacher and supported my passion for music from a young age. I am forever grateful for her guidance and encouragement.', "### Human: What is your interest?\n### Assistant: My interest lies in music, my dear sir. It is my passion and my life's work. I have dedicated myself to the art of composing and performing music, and I find great joy in sharing my creations with the world."]}


In [21]:
dataset[0]

{'text': '### Human: What do you think of your father?\n### Assistant: My father was a strict man, but he was also my first music teacher. He recognized my talent at a young age and pushed me to develop my skills. I am grateful for his guidance, but our relationship was not always easy.'}

In [22]:
dataset.push_to_hub("beethoven-qa-merged")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/HamdanXI/beethoven-qa-merged/commit/83ec33c80578292299394cd7e59f7b461307db45', commit_message='Upload dataset', commit_description='', oid='83ec33c80578292299394cd7e59f7b461307db45', pr_url=None, pr_revision=None, pr_num=None)

In [31]:
# Define a function to transform the data
def transform_conversation(example):
    conversation_text = example['text']
    segments = conversation_text.split('###')

    reformatted_segments = []

    # Iterate over pairs of segments
    for i in range(1, len(segments) - 1, 2):
        human_text = segments[i].strip().replace('Human:', '').strip()

        # Check if there is a corresponding assistant segment before processing
        if i + 1 < len(segments):
            assistant_text = segments[i+1].strip().replace('Assistant:', '').strip()

            # Apply the new template
            reformatted_segments.append(f'<s>[INST] {human_text} [/INST] {assistant_text} </s>')
        else:
            # Handle the case where there is no corresponding assistant segment
            reformatted_segments.append(f'<s>[INST] {human_text} [/INST] </s>')

    return {'text': ''.join(reformatted_segments)}


# Apply the transformation
transformed_dataset = dataset.map(transform_conversation)

Map:   0%|          | 0/1522 [00:00<?, ? examples/s]

In [32]:
transformed_dataset

Dataset({
    features: ['text'],
    num_rows: 1522
})

In [34]:
transformed_dataset.push_to_hub("beethoven-qa-merged-for-tinyllama")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/HamdanXI/beethoven-qa-merged-for-tinyllama/commit/d9ee84ece794d9cbaf2edf39187c277428f39e75', commit_message='Upload dataset', commit_description='', oid='d9ee84ece794d9cbaf2edf39187c277428f39e75', pr_url=None, pr_revision=None, pr_num=None)